In [1]:
from pymongo import MongoClient
from datetime import datetime, timedelta
import random
import pandas as pd
from names_generator import generate_name
import os
import json
from bson import ObjectId

In [4]:

# Connect to MongoDB server
client = MongoClient("mongodb://localhost:27017/")

# Access the 'class_2' database
db = client["ecommerce"]



### create and update the customers collection

In [3]:
# Access the 'ecommerce' collection
collection = db["customers"]

def generate_customers():
    """Generate a list of customer documents with structured addresses."""
    streets = [
        'Broad Street', 'Shehu Shagari Way', 'Abraham Adesanya Street',
        'Nigerian Port Authority Road', 'Ring Road', 'Nnamdi Azikiwe Avenue',
        'Kawo Road', 'Bode Thomas Street', 'Ajose Adeogun Street'
    ]
    cities_states = {
        "Owerri": "Imo", "Abuja": "FCT", "Ibadan": "Oyo", 
        "Port Harcourt": "Rivers", "Kano": "Kano"
    }
    
    customers = []
    for i in range(20):
        selected_city = random.choice(list(cities_states.keys()))
        selected_state = cities_states[selected_city]
        
        new_customer = {
            "_id": ObjectId(),
            "customer_id": i + 1,
            "name": f"Customer {i + 1}",
            "email": f"customer{i + 1}@randle.com",
            "phone": f"{random.randint(100, 999)}-{random.randint(100, 999)}-{random.randint(1000, 9999)}",
            "address": {
                "street": random.choice(streets),
                "city": selected_city,
                "state": selected_state
            }
        }
        customers.append(new_customer)
    
    return customers


In [ ]:
def setup_customer_data():
    """Main function to set up customer data in MongoDB."""
    try:
        print("Connecting to MongoDB...")
        
        # Connect to MongoDB
        client = MongoClient('mongodb://localhost:27017/')
        db = client['ecommerce']
        
        # Generate and insert customer data
        customers = generate_customers()
        print("Dropping existing 'customers' collection...")
        db.customers.drop()
        
        print("Inserting new customer data...")
        db.customers.insert_many(customers)
        
        # Verify insertion
        print("Fetching sample customer document...")
        sample_customer = db.customers.find_one()
        print("\nSample customer document:")
        print(json.dumps(sample_customer, indent=2, default=str))
    except Exception as e:
        print(f"An error occurred: {e}")

# Call the setup_customer_data function to insert the data
setup_customer_data()


### create and update the products collection

In [ ]:

collection = db["products"]

def sample_products(num_products=20):
    """Generate sample product data"""
    products = []
    categories = ["Electronics", "Clothing", "Books", "Home & Garden", "Sports"]
    products_by_category = {
        "Electronics": ["Laptop", "Phone", "Tablet", "Headphones", "Smart Watch"],
        "Clothing": ["T-Shirt", "Jeans", "Jacket", "Shoes", "Hat"],
        "Books": ["Novel", "Textbook", "Comic Book", "Biography", "Cookbook"],
        "Home & Garden": ["Plant Pot", "Garden Tools", "Furniture", "Lighting", "Decorations"],
        "Sports": ["Basketball", "Tennis Racket", "Running Shoes", "Yoga Mat", "Weights"]
    }
    
    for i in range(num_products):
        category = random.choice(categories)
        products.append({
            "_id": ObjectId(),
            "product_id": 101 + i,
            "product_name": f"{random.choice(products_by_category[category])} ",
            "category": category,
            "price": round(random.uniform(10, 2000), 2),
            "stock": random.randint(0, 100)
        })
    
    return products

# Generate sample products
products = sample_products()

In [ ]:
# Create a DataFrame from the products list
df = pd.DataFrame(products)

# Writing to CSV
df.to_csv('products.csv', index=False)

df

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client["ecommerce"]
collection = db['products']

# Read CSV file
df = pd.read_csv('C:/Users/phili/Downloads/products.csv')

# Convert DataFrame to list of dictionaries
data = df.to_dict(orient='records')

# Insert data into MongoDB
products = collection.insert_many(data)

print(f"Inserted {len(products.inserted_ids)} documents into the collection.")

### create and update the orders collection

In [ ]:

collection = db["orders"]

def sample_orders(num_orders=20, num_customers=20):
    """Generate sample order data"""
    orders = []
    statuses = ["Pending", "Processing", "Shipped", "Delivered", "Cancelled"]
    
    for i in range(5001, 5001 + num_orders):
        order_date = datetime.now() - timedelta(days=random.randint(0, 365))
        delivery_date = order_date + timedelta(days=random.randint(1, 14))  
        
        orders.append({
            "_id": ObjectId(),
            "order_id": i,
            "customer_id": random.randint(1, num_customers),
            "order_date": order_date,  # Keeping datetime format
            "status": random.choice(statuses),
            "delivery_date": delivery_date  # Keeping datetime format
        })
    
    return orders

# Generate sample orders
orders = sample_orders()

In [ ]:
# Create a DataFrame from the products list
df = pd.DataFrame(orders)

# Writing to CSV
df.to_csv('orders.csv', index=False)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client["ecommerce"]
collection = db['orders']

# Read CSV file
df = pd.read_csv('C:/Users/phili/Downloads/orders.csv')

# Convert DataFrame to list of dictionaries
data = df.to_dict(orient='records')

# Insert data into MongoDB
orders = collection.insert_many(data)

print(f"Inserted {len(orders.inserted_ids)} documents into the collection.")

### create and update the products collection

In [ ]:
def sample_order_items(orders, products, num_items_per_order=3):
    """Generate sample order items data"""
    order_item = []
    order_item_id = 9001
    
    for order in orders:
        selected_products = random.sample(products, random.randint(1, num_items_per_order))
        
        for product in selected_products:
            order_item.append({
                "_id": ObjectId(),
                "order_item_id": order_item_id,
                "order_id": order["order_id"],
                "product_id": product["product_id"],
                "quantity": random.randint(1, 5),
                "price": product["price"]
            })
            order_item_id += 1
    
    return order_item

# Fetch the orders and products from MongoDB to make sure they are lists of dictionaries
orders = list(db.orders.find())
products = list(db.products.find())

# Generate sample order items
order_item = sample_order_items(orders, products)

# Print sample order items to verify
for item in order_item[:5]:  # Print first 5 items for brevity
    print(item)


In [ ]:
# Create a DataFrame from the order items list
df = pd.DataFrame(order_item)

# Writing to CSV
df.to_csv('order_item.csv') #index=False)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client["ecommerce"]
collection = db['order_item']

# Read CSV file
df = pd.read_csv('C:/Users/phili/Downloads/order_item.csv')

# Convert DataFrame to list of dictionaries
data = df.to_dict(orient='records')

# Insert data into MongoDB
order_item = collection.insert_many(data)

print(f"Inserted {len(order_item.inserted_ids)} documents into the collection.")

# Determine Revenue be Category, sorting in descending order

In [ ]:
def get_revenue_by_category(db):
    """Calculate revenue by product category"""
    pipeline = [
        {
            "$lookup": {
                "from": "order_item",
                "localField": "product_id",
                "foreignField": "product_id",
                "as": "sales"
            }
        },
        {"$unwind": "$sales"},
        {
            "$group": {
                "_id": "$category",
                "total_revenue": {
                    "$sum": {"$multiply": ["$sales.quantity", "$sales.price"]}
                },
                "number_of_orders": {"$sum": 1}
            }
        },
        {"$sort": {"total_revenue": -1}}
    ]
    
    results = list(db.products.aggregate(pipeline))
    return results

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['ecommerce']

# Call the function
revenue_by_category = get_revenue_by_category(db)

# Print the results
for category in revenue_by_category:
    print(f"Category: {category['_id']}, Total Revenue: {category['total_revenue']}, Number of Orders: {category['number_of_orders']}")


# Calculate the difference between order_date and delivery_date.

In [ ]:
# Define the aggregation pipeline
pipeline = [
    {
        "$addFields": {
            "order_date": { "$toDate": "$order_date" },
            "delivery_date": { "$toDate": "$delivery_date" }
        }
    },
    {
        "$project": {
            "order_date": 1,
            "delivery_date": 1,
            "delivery_time_in_days": {
                "$divide": [
                    { "$subtract": ["$delivery_date", "$order_date"] }, 86400000  # Convert milliseconds to days
                ]
            }
        }
    }
]


In [ ]:
# Execute the aggregation pipeline
result_cursor = db.orders.aggregate(pipeline)

# Convert the cursor to a list and create a DataFrame
result = list(result_cursor)
df = pd.DataFrame(result)

# Print the DataFrame
print("Delivery time in days for each order:")
print(df)

# Calculate the average delivery time.

In [ ]:
# Define the aggregation pipeline
pipeline = [
    {
        "$addFields": {
            "order_date": { "$toDate": "$order_date" },
            "delivery_date": { "$toDate": "$delivery_date" }
        }
    },
    {
        "$project": {
            "order_date": 1,
            "delivery_date": 1,
            "delivery_time": {
                "$subtract": ["$delivery_date", "$order_date"]
            }
        }
    },
    {
        "$group": {
            "_id": None,
            "average_delivery_time": {
                "$avg": "$delivery_time"
            }
        }
    },
    {
        "$project": {
            "average_delivery_time_in_days": {
                "$divide": ["$average_delivery_time", 86400000]  # Convert milliseconds to days
            }
        }
    }
]




In [ ]:
# Execute the aggregation pipeline
result_cursor = db.orders.aggregate(pipeline)

# Convert the cursor to a list and print the results
result = list(result_cursor)
print("Average delivery time in days:")
for item in result:
    print(item)

# Count customers by state, sorting in descending order

In [ ]:
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")  
db = client["ecommerce"]  
customers = db["customers"]  

# Define the pipeline
pipeline = [
    {
        "$group": {
            "_id": {
                "state": "$address.state",
                "city": "$address.city"
            },
            "count": {"$sum": 1}
        }
    },
    {
        "$sort": {"count": -1}
    }
]



In [ ]:
# Execute the pipeline
result = list(customers.aggregate(pipeline))

# Print results
for record in result:
    state = record["_id"]["state"]
    city = record["_id"]["city"]
    count = record["count"]
    print(f"State: {state}, City: {city}, Count: {count}")


 # top 3 most expensive products sold in each order

In [ ]:



# Define the aggregation pipeline
pipeline = [
    {
        "$lookup": {
            "from": "products",
            "localField": "product_id",
            "foreignField": "product_id",
            "as": "product_details"
        }
    },
    { "$unwind": "$product_details" },
    {
        "$sort": {
            "product_details.price": -1
        }
    },
    {
        "$group": {
            "_id": "$_id",
            "top_products": {
                "$push": "$product_details"
            }
        }
    },
    {
        "$project": {
            "_id": 1,
            "top_products": {
                "$slice": ["$top_products", 3]
            }
        }
    }
]




In [ ]:
# Execute the aggregation pipeline
results = db.orders.aggregate(pipeline)

# Print the results
print("Top products in each order:")
for result in results:
    print(f"Order ID: {result['_id']}")
    for product in result['top_products']:
        print(f"  Product: {product['name']}, Price: {product['price']}")

# embedded and referenced schema designs 

1. Customers 
Embedded Schema Design:

Rationale: Each customer has a unique address that does not need to be shared with other customers. Embedding the address within the customer document simplifies access and updates.

Example:

{
    "_id": ObjectId("67683bc3091b856eb9814708"),
    "customer_id": 1,
    "name": "Customer 1",
    "email": "customer1@randle.com",
    "phone": "222-990-5354",
    "address": {
        "street": "Ring Road",
        "city": "Ibadan",
        "state": "Oyo"
    }
}

2. Products
Flat Schema Design:

Rationale: Products generally do not have sub-documents or nested data that would benefit from embedding. Each product can be stored as a flat document.

Example:

{
    "_id": ObjectId(),
    "product_id": 101,
    "product_name": "Laptop",
    "category": "Electronics",
    "price": 999.99,
    "stock": 50
}


3. Orders and Order Items
Referenced Schema Design:

Rationale: Orders often contain multiple order items, and embedding all order items within an order document could lead to large document sizes. Using references allows you to link order items to orders without duplication, maintaining a scalable and normalized structure.

Order Example:


{
    "_id": ObjectId(),
    "order_id": 5001,
    "customer_id": 1,
    "order_date": ISODate("2023-12-01T10:00:00Z"),
    "status": "Delivered",
    "delivery_date": ISODate("2023-12-14T10:00:00Z")
}

Order Item Example:

json
{
    "_id": ObjectId(),
    "order_item_id": 9001,
    "order_id": 5001,
    "product_id": 101,
    "quantity": 2,
    

# Create indexes on frequently queried fields like customer_id and product_id.

In [ ]:
db.customers.create_index([("customer_id", 1)], unique=True)
db.products.create_index([("product_id", 1)], unique=True)
db.orders.create_index([("order_id", 1)], unique=True)
db.orders.create_index([("customer_id", 1)])
db.order_items.create_index([("order_id", 1)])
db.order_items.create_index([("product_id", 1)])

# Simulate an order creation process that updates the orders and inventory collections atomically.

In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['ecommerce']



In [ ]:
from pymongo import MongoClient
from datetime import datetime, timedelta
from bson import ObjectId
import random

def create_order(db, customer_id, product_orders):
    """
    Create a new order with multiple products.
    
    Args:
        db: MongoDB database connection
        customer_id: ID of the customer placing the order
        product_orders: List of dicts with product_id and quantity
        
    Returns:
        tuple: (success (bool), message (str), order_id (int))
    """
    try:
        # Validate customer exists
        customer = db.customers.find_one({"customer_id": customer_id})
        if not customer:
            return False, f"Customer not found: customer_id {customer_id}", None
            
        # Pre-validate all products and check stock
        products_to_update = []
        for product_order in product_orders:
            product_id = product_order.get("product_id")
            quantity = product_order.get("quantity")
            
            # Validate input data
            if not product_id or not quantity:
                return False, "Invalid product order format", None
            if quantity <= 0:
                return False, f"Invalid quantity for product_id {product_id}", None
                
            # Check product exists and has sufficient stock
            product = db.products.find_one({"product_id": product_id})
            if not product:
                return False, f"Product not found: product_id {product_id}", None
            if product["stock"] < quantity:
                return False, f"Insufficient stock for product_id {product_id}. Available: {product['stock']}, Requested: {quantity}", None
                
            products_to_update.append({
                "product": product,
                "quantity": quantity
            })
        
        # Start a session for transaction
        with db.client.start_session() as session:
            with session.start_transaction():
                # Generate unique order_id
                last_order = db.orders.find_one(sort=[("order_id", -1)])
                order_id = (last_order["order_id"] + 1) if last_order else 1
                
                # Create order document
                order = {
                    "order_id": order_id,
                    "customer_id": customer_id,
                    "order_date": datetime.now(),
                    "status": "Processing",
                    "delivery_date": datetime.now() + timedelta(days=random.randint(1, 14))
                }
                
                # Insert order
                db.orders.insert_one(order, session=session)
                
                # Process each product order
                total_amount = 0
                for product_update in products_to_update:
                    product = product_update["product"]
                    quantity = product_update["quantity"]
                    
                    # Create order item
                    order_item = {
                        "order_item_id": ObjectId(),
                        "order_id": order_id,
                        "product_id": product["product_id"],
                        "quantity": quantity,
                        "price": product["price"],
                        "subtotal": product["price"] * quantity
                    }
                    
                    # Insert order item
                    db.order_items.insert_one(order_item, session=session)
                    total_amount += order_item["subtotal"]
                    
                    # Update product stock
                    db.products.update_one(
                        {"product_id": product["product_id"]},
                        {"$inc": {"stock": -quantity}},
                        session=session
                    )
                
                # Update order with total amount
                db.orders.update_one(
                    {"order_id": order_id},
                    {"$set": {"total_amount": total_amount}},
                    session=session
                )
                
                return True, f"Order {order_id} created successfully", order_id
                
    except Exception as e:
        return False, f"Error creating order: {str(e)}", None


In [ ]:
def test_create_order():
    product_orders = [
        {"product_id": 101, "quantity": 2},
        {"product_id": 102, "quantity": 1}
    ]
    
    client = MongoClient('mongodb://localhost:27017/')
    db = client['ecommerce']
    
    success, message, order_id = create_order(db, customer_id=1, product_orders=product_orders)
    print(f"Success: {success}")
    print(f"Message: {message}")
    print(f"Order ID: {order_id}")

# Run the test function
test_create_order()


# Monitor real-time changes in the orders collection.

In [ ]:
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['ecommerce']


In [ ]:
def monitor_orders():
    with db.orders.watch() as stream:
        print("Watching for changes in the 'orders' collection...")
        for change in stream:
            print(f"Change detected: {change}")

# Call the function to start monitoring
monitor_orders()


# Ensure all documents in the products collection include a valid price.

In [ ]:
from pymongo import MongoClient
from bson import ObjectId

def ensure_valid_prices():
    # Connect to MongoDB
    client = MongoClient('mongodb://localhost:27017/')
    db = client['ecommerce']
    products_collection = db['products']

    # Define the criteria for invalid prices (missing or not a positive number)
    invalid_criteria = {
        "$or": [
            {"price": {"$exists": False}},
            {"price": {"$lte": 0}}
        ]
    }

    # Find products with invalid prices
    invalid_products = products_collection.find(invalid_criteria)
    for product in invalid_products:
        print(f"Invalid product found: {product}")

        # Update the product with a valid price (setting a default price of 100.00)
        products_collection.update_one(
            {"_id": product["_id"]},
            {"$set": {"price": 100.00}}
        )

    print("All products have valid prices now.")

# Call the function to ensure all products have valid prices
ensure_valid_prices()
